Show a Ham Email

In [10]:
corpus_path = 'C:/Users/Rik/Documents/corpus/'
file_path = corpus_path + 'enron1/ham/0007.1999-12-14.farmer.ham.txt'
with open(file_path, 'r') as infile:
    ham_sample = infile.read()
    print(ham_sample)

Subject: mcmullen gas for 11 / 99
jackie ,
since the inlet to 3 river plant is shut in on 10 / 19 / 99 ( the last day of
flow ) :
at what meter is the mcmullen gas being diverted to ?
at what meter is hpl buying the residue gas ? ( this is the gas from teco ,
vastar , vintage , tejones , and swift )
i still see active deals at meter 3405 in path manager for teco , vastar ,
vintage , tejones , and swift
i also see gas scheduled in pops at meter 3404 and 3405 .
please advice . we need to resolve this as soon as possible so settlement
can send out payments .
thanks


Show a Spam Email

In [11]:
file_path = corpus_path + 'enron1/spam/0058.2003-12-21.GP.spam.txt'
with open(file_path, 'r') as infile:
    spam_sample = infile.read()
    print(spam_sample)

Subject: stacey automated system generating 8 k per week parallelogram
people are
getting rich using this system ! now it ' s your
turn !
we ' ve
cracked the code and will show you . . . .
this is the
only system that does everything for you , so you can make
money
. . . . . . . .
because your
success is . . . completely automated !
let me show
you how !
click
here
to opt out click here % random _ text



Read in ham and spam email

In [12]:
import glob
import os

In [14]:
emails, labels = [], []

In [15]:
def readFilesForFolder(folder_path, is_spam):
    file_path = corpus_path + 'enron1/' + folder_path
    for filename in glob.glob(os.path.join(file_path, '*.txt')):
        with open(filename, 'r', encoding="ISO-8859-1") as infile:
            emails.append(infile.read())
            labels.append(is_spam)

In [16]:
readFilesForFolder('spam', 1)
print(len(emails))
print(len(labels))
readFilesForFolder('ham', 0)
print(len(emails))
print(len(labels))

1500
1500
5172
5172


Preprocess and clean raw text data

In [18]:
from nltk.corpus import names
from nltk.stem import WordNetLemmatizer
def is_letter_only(word):
    return word.isalpha()
all_names = set(names.words())
lemmatizer = WordNetLemmatizer()
def clean_text(docs):
    docs_cleaned = []
    for doc in docs:
        doc = doc.lower()
        doc_cleaned = ' '.join(lemmatizer.lemmatize(word) for word in doc.split() if is_letter_only(word))
        docs_cleaned.append(doc_cleaned)
    return docs_cleaned
emails_cleaned= clean_text(emails)

Create CountVectorizer

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words="english", max_features=1000, max_df=0.5, min_df=2)
docs_cv = cv.fit_transform(emails_cleaned)

  (0, 265)	2
  (0, 482)	2
  (0, 363)	2
  (0, 229)	1
  (0, 361)	2
  (0, 103)	1
  (0, 997)	1
  (0, 86)	2
  (0, 72)	1
  (0, 476)	1
  (0, 897)	1
  (0, 691)	1
  (0, 506)	1
  (0, 864)	1
  (0, 585)	1
  (0, 151)	1
  (0, 715)	1
  (0, 968)	1
  (0, 932)	1


Investigate first doc
sparse vector is in the form of the following:
(row index, term index) term_frequency

In [34]:
print(docs_cv[0])

  (0, 265)	2
  (0, 482)	2
  (0, 363)	2
  (0, 229)	1
  (0, 361)	2
  (0, 103)	1
  (0, 997)	1
  (0, 86)	2
  (0, 72)	1
  (0, 476)	1
  (0, 897)	1
  (0, 691)	1
  (0, 506)	1
  (0, 864)	1
  (0, 585)	1
  (0, 151)	1
  (0, 715)	1
  (0, 968)	1
  (0, 932)	1


Corresponding terms can be found through feature names

In [36]:
terms = cv.get_feature_names()
print(terms[932])
print(terms[968])
print(terms[715])

unsubscribe
website
read


Group the data by label and record the index of samples

In [37]:
def get_label_index(labels):
    from collections import defaultdict
    label_index = defaultdict(list)
    for index, label in enumerate(labels):
        label_index[label].append(index)
    return label_index
label_index = get_label_index(labels)

Calculate Prior

In [39]:
def get_prior(label_index):
    """
    Compute prior based on training samples
    @param label_index: grouped sample indices by class
    @return: dictionary, with class label as key, corresponding prior as the value
    """
    prior = {label: len(index) for label, index in label_index.items()}
    total_count = sum(prior.values())
    for label in prior:
        prior[label] /= float(total_count)
    return prior
prior = get_prior(label_index)
print("Prior:", prior)

Prior: {1: 0.2900232018561485, 0: 0.7099767981438515}


Calculate Likelihood

In [40]:
import numpy as np
def get_likelihood(term_matrix, label_index, smoothing=0):
    """
    Compute likelihood based on training samples
    @param term_matrix: sparse matrix of the term frequency features
    @param label_index: grouped sample indices by class
    @param smoothing: integer, additive Laplace smoothing parameter
    @return: dictionary, with class as key, corresponding conditional probability P(feature|class) vector as value
    """
    likelihood = {}
    for label, index in label_index.items():
        likelihood[label] = term_matrix[index, :].sum(axis=0) + smoothing
        likelihood[label] = np.asarray(likelihood[label])[0]
        total_count = likelihood[label].sum()
        likelihood[label] = likelihood[label] / float(total_count)
    return likelihood
smoothing = 1
likelihood = get_likelihood(docs_cv, label_index, smoothing)
len(likelihood[0])

1000

In [42]:
print(likelihood[0][:5])
print(likelihood[1][:5])

[0.00024653 0.00090705 0.00080007 0.00032096 0.00073495]
[0.00063304 0.00078026 0.00101581 0.00022083 0.00326826]


Get Posterior

In [43]:
def get_posterior(term_matrix, prior, likelihood):
    """
    Compute posterior of testing samples, based on prior and likelihood
    @param term_matrix: sparse matrix of the term frequency features
    @param prior: dictionary, with class label as key,
                    corresponding to prior as the value
    @param likelihood: dictionary, with class label as key,
                    corresponding conditional probability vector as value
    @return: dictionary, with class label as key, corresponding posterior as value
    """
    num_docs = term_matrix.shape[0]
    posteriors = []
    for i in range(num_docs):
        # posterior is proportional to prior * likelihood
        # = exp(log(prior * likelihood))
        # = exp(log(prior) + log(likelihood))
        posterior = { key: np.log(prior_label) for key, prior_label in prior.items() }
        for label, likelihood_label in likelihood.items():
            term_document_vector = term_matrix.getrow(i)
            counts = term_document_vector.data
            indices = term_document_vector.indices
            for count, index in zip(counts, indices):
                posterior[label] += np.log(likelihood_label[index]) * count
        # exp(-1000):exp(-999) will cause division error,
        # however it equates to exp(0):exp(1)
        min_log_posterior = min(posterior.values())
        for label in posterior:
            try:
                posterior[label] = np.exp(posterior[label] - min_log_posterior)
            except:
                posterior[label] = float('inf')
        # normalize so that all sums up to 1
        sum_posterior = sum(posterior.values())
        for label in posterior:
            if posterior[label] == float('inf'):
                posterior[label] = 1.0
            else:
                posterior[label] /= sum_posterior
        posteriors.append(posterior.copy())
    return posteriors

Test emails

In [44]:
emails_test = [
    '''Subject: flat screens
    hello,
    please call or contact regarding the other flat screens
    requested .
    trisha tlapek - eb 3132 b
    michael sergeev - eb 3132 a
    also the sub blocker that was taken away from eb 3131 a .
    trisha should two monitors also michael .
    thanks
    kevin moore''',
    '''Subject: let ' s stop the mlm insanity !
    still believe you can earn $ 100 , 000 fast in mlm ? get real !
    get emm , a brand new system that replaces mlm with something that works !
    start earning 1 , 000 ' s now ! up to $ 10 , 000 per week doing simple online tasks .
    free info - breakfree @ luxmail . com - type " send emm info " in the
    subject box .
    this message is sent in compliance of the proposed bill section 301 . per
    section 301 , paragraph ( a ) ( 2 ) ( c ) of s . 1618 . further transmission
    to you by the sender of this e - mail may be stopped at no cost to you by
    sending a reply to : " email address " with the word remove in the line .'''
]

In [45]:
emails_cleaned_test = clean_text(emails_test)
term_docs_test = cv.transform(emails_cleaned_test)
posterior = get_posterior(term_docs_test, prior, likelihood)
print(posterior)

[{1: 3.47152510287432e-09, 0: 0.9999999965284749}, {1: 0.999999999999999, 0: 9.596751608722133e-16}]


Split data in train and test data

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(emails_cleaned, labels, test_size=0.33, random_state=42)
print(len(X_train), len(Y_train))
print(len(X_test), len(Y_test))

3465 3465
1707 1707


Train just training data

In [60]:
term_docs_train = cv.fit_transform(X_train)
label_index = get_label_index(Y_train)
prior = get_prior(label_index)
likelihood = get_likelihood(term_docs_train, label_index, smoothing)

Predict posterior for testing data

In [61]:
term_docs_test = cv.transform(X_test)
posterior = get_posterior(term_docs_test, prior, likelihood)

c:\python37\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in exp


Evaluate performance

In [63]:
correct = 0.0
for pred, actual in zip(posterior, Y_test):
    if actual == 1:
        if pred[1] >= 0.5:
            correct += 1
    elif pred[0] > 0.5:
        correct += 1
print('The accuracy on {0} testing samples is : {1:.1f}%'.format(len(Y_test), correct/len(Y_test)*100))

The accuracy on 1707 testing samples is : 93.0%
